In [2]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_iris

mnist = tf.keras.datasets.mnist 
(x_mnist_train, y_mnist_train), (x_mnist_test, y_mnist_test) = mnist.load_data()
x_mnist_train, x_mnist_test = x_mnist_train / 255.0, x_mnist_test / 255.0
train_images = x_mnist_train.reshape((60000, 28, 28, 1))
train_labels = y_mnist_train.reshape((60000, 1))
test_images = x_mnist_test.reshape((10000, 28, 28, 1))
test_labels = y_mnist_test.reshape((10000, 1))

def train(model, epochs=1):
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(train_images, train_labels, epochs=epochs, validation_data=(test_images, test_labels))

## 5-2. スキップ接続
ここまで、リカレントニューラルネット以外では基本的には

![alt](forward.jpg)

のような**順番に伝搬してゆく** ネットワーク構造を考えていましたが、ニューラルネットワークをそのようにしか作ってはいけないわけではありません。勾配の自動計算にはグラフ構造があれば十分なので、

![alt](skip.jpg)

のようなネットワーク構造を考えても良いのです。このような構造は**スキップ接続(skip connections)** と呼ばれます。まず簡単なスキップ接続として

$$
h = \sigma(z) + z
$$

を考えてみます。ここで $\sigma$ は何らかの非線形関数 (sigmoidやrelu)とします。この出力 $h$ の 入力 $z$ についての微分値は当たり前ですが

$$
\frac{d h}{dz} = \sigma'(z) + 1
$$

となります。$\sigma$ がsigmoid関数やrelu関数の場合、スキップなしの場合 $\sigma'(z)$ はすごく小さい値になりがちですが、スキップ有りの場合、これに $+1$ が加わるので、勾配が大きくなります。誤差関数 $L$ を $h$ の関数と見ると、

$$
\frac{dL}{dz} = \frac{dL}{dh} \frac{d h}{dz}
$$

なので、誤差関数の勾配も大きくなってSGD訓練がはかどりそうです。とくにreluの場合、オフセットよりも小さな値は**ゼロ** なので、順番にベクトルを重ねるネットワークだと**情報が落ちていってしまいます**。それに比べ、スキップ接続は入力の情報量を減らしません。ここでは幾つかのスキップ接続を紹介します。

### Residual network
スキップ接続を持つネットワークで有名なのが [arXiv:1512.03385](https://arxiv.org/abs/1512.03385) の**残差ネットワーク(Residual network, ResNet)** です。これは超深層ネットワーク（**~$10^{2 \sim 3}$層**）の訓練のために導入された**残差ブロック(Residual Block, ResBlock)** という構成要素(モジュール)を新たに提唱した論文でもあります。超深層ではない場合にも残差ブロックを使ったネットワークがしばしば使われますので、ここで説明しておいて損はないでしょう。

#### 素朴なResBlock
論文では畳み込みを前提としたResBlockを使っていますが、
まずは畳込みではなく、ベクトルを処理してゆく形式で最も単純なResBlockを作ってみます：

![alt](resblock1.jpg)

$$
{\bf y} = \sigma({\color{red}{W}}{\bf x} + {\color{red}{\bf b}})  + {\bf x}
$$

の形のものです。この処理1ブロックを新たなネットワークの「部品」と考えます。tenforflowでは `tf.keras.layers.Layer` を継承したクラスを定義すれば他の便利機能と一緒に使えるので便利です。定義には
* `__init__(self)`: クラス初期化関数
* `call(self, inputs)`: 層の入力 `inputs` が入ってきたときの出力

が最低限あれば良いです。例えば以下のように書けばよいです：

In [3]:
class DenseResBlock1_pre(tf.keras.layers.Layer):
    def __init__(self, units):
        super(DenseResBlock1_pre, self).__init__()
        self.f = tf.keras.layers.Dense(units, activation='sigmoid')        
    
    def call(self, inputs):
        return self.f(inputs) + inputs
    
x = tf.random.uniform(shape=[1, 5])
ResBlock = DenseResBlock1_pre(units=5)
ResBlock(x)

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[1.2919633 , 0.94248337, 0.7971257 , 1.4403968 , 0.5051593 ]],
      dtype=float32)>

上の例では 5次元ベクトル ${\bf x}$ に作用させるResBlockを書きました。ここで実装上のテクニックなのですが、${\bf y} = \sigma({\color{red}{W}}{\bf x} + {\color{red}{\bf b}})  + {\bf x}$ のベクトル和が定義できるためには、用意する ${\color{red}{W, {\bf b}}}$ による線形変換が ${\bf x}$ と同じ次元のベクトルを返さねばなりません。このため、上の実装で `x = tf.random.uniform(shape=[1, 5])`となっているところを適当な別のshapeに取るとエラーが出てしまいます。ここの処理は実は **自動化** できます。それをするには
* `__init__(self)` にはパラメータ読み込みを書かず、super関数による初期化だけ書く
* 訓練パラメータは代わりに`build(self, input_shape)`で定義する

とすれば良いです。この`build`というのはそのLayerオブジェクトが呼ばれて実際にベクトルの処理をしたときに実行される関数ですので、入寮ベクトルのサイズをわざわざ指定しなくても良くなります：

In [4]:
class DenseResBlock1(tf.keras.layers.Layer):
    def __init__(self):
        super(DenseResBlock1, self).__init__()
        
    def build(self, input_shape):
        units = input_shape[-1] # input_shape = (batchsize, dim_of_vector), and input_shape[-1] = dim_of_vector
        self.f = tf.keras.layers.Dense(units, activation='sigmoid')
    
    def call(self, inputs):
        return self.f(inputs) + inputs
    
x = tf.random.uniform(shape=[1, 6])
ResBlock = DenseResBlock1()
ResBlock(x)

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[1.1218925 , 0.3582188 , 0.8069558 , 1.5057303 , 0.89166486,
        1.4005613 ]], dtype=float32)>

#### 素朴なResBlockその2
上の例では ${\bf x}$ と $\sigma(W{\bf x} + {\bf b})$ をベクトルとして足し合わせるために同じ次元にしなければいけなかったわけですが、必ずしもそういうわけではなく、以下のようなスキップ構造を考えることもできます：

![alt](resblock2.jpg)

$$
{\bf y} = \sigma({\color{red}{W_1}}{\bf x} + {\color{red}{\bf b}})  + {\color{red}{W_2}}{\bf x}
$$

これも作れます。こちらは出力先を指定すればよいので`build`関数は必要ないです（というか`tf.keras.layers.Dense`に`build`が定義されているのだと思います）。

In [5]:
class DenseResBlock2(tf.keras.layers.Layer):
    def __init__(self, units):
        super(DenseResBlock2, self).__init__()
        self.f1 = tf.keras.layers.Dense(units, activation='sigmoid')
        self.w2 = tf.keras.layers.Dense(units, use_bias=False)
    
    def call(self, inputs):
        return self.f1(inputs) + self.w2(inputs)
    
x = tf.random.uniform(shape=[1, 6])
ResBlock = DenseResBlock2(units=3)
ResBlock(x)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.02537549, 0.5109363 , 1.3208609 ]], dtype=float32)>

#### 定義したblockを使う
いままでの`tf.keras.layers.Layer`クラスと同じように使えます。例えばアヤメデータの訓練をResNetでやってみます：

> 注意ですが、一層目での`input_shape`の引数をここまでのResBlockでは定義して来ませんでした。これは無くても動く（つまり以下のコードで一層目を削除しても訓練は進む）ようですが、`model.summary()`で上手く中間での出力を出してくれません。このあたりまで自動化する場合はやはりBlackの`build()`関数にその処理を書くべきなのだと思います。

以下のようなモデルを作ってみました。

![alt](resnet.jpg)

In [6]:
iris = load_iris() # アヤメデータ読み込み
X_train, Y_train = iris.data, iris.target

model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(10, input_shape=(4,), activation='relu'),
                                    DenseResBlock1(),          # ここに挟んだ
                                    DenseResBlock2(units=3), # ここに挟んだ
                                    tf.keras.layers.Dense(3, activation='softmax')
                                   ])
model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
hist = model.fit(X_train, Y_train, epochs=5, batch_size=10, validation_split=0.2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_res_block1_1 (DenseRes (None, 10)                110       
_________________________________________________________________
dense_res_block2_1 (DenseRes (None, 3)                 63        
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 12        
Total params: 235
Trainable params: 235
Non-trainable params: 0
_________________________________________________________________
Train on 120 samples, validate on 30 samples
Epoch 1/5
120/120 [==============================] - 1s 4ms/sample - loss: 0.9740 - val_loss: 1.7219
Epoch 2/5
120/120 [==============================] - 0s 415us/sample -

#### 素朴な畳み込みResBlock
畳み込み演算に対するResBlockはチャネル方向を一単位にしてスキップ構造を作ります。ベクトルの成分をそのままチャネル成分に置き換えて図を描くと

![alt](resblock3.jpg)

こんな感じになります。`Dense`を`Conv2D`に置き換えて、画像サイズを保つようにしておけば同じように実装できます：


In [3]:
class MyResBlock(tf.keras.layers.Layer):
    def __init__(self, kernel_size, activation):
        super(MyResBlock, self).__init__()
        self.kernel_size = kernel_size # 
        self.activation = activation   # ここではじめに設定を読み込んでおくとよい
        
    def build(self, input_shape):
        channels = input_shape[-1] # input_shape = (batchsize, Lx, Ly, channels), and input_shape[-1] = channels
        self.f = tf.keras.layers.Conv2D(channels, self.kernel_size, padding='same', activation=self.activation)
    
    def call(self, inputs):
        return self.f(inputs) + inputs
    
x = tf.random.uniform(shape=[2, 28, 28, 3])
ResBlock = MyResBlock((3,3), activation='relu')
ResBlock(x).shape

TensorShape([2, 28, 28, 3])

#### 論文のResBlockその1
論文 [arXiv:1512.03385](https://arxiv.org/abs/1512.03385) で導入されたResBlockは2つあります。まず最初のものは

![alt](resblock4.jpg)

のように、スキップ内部に 畳み込み-relu-畳み込みを挟んで、最後に合算した後ふたたびrelu、というものです：

In [4]:
class ResBlock1(tf.keras.layers.Layer):
    def __init__(self, kernel_size=(3,3)):
        super(ResBlock1, self).__init__()
        self.kernel_size = kernel_size # 
        
    def build(self, input_shape):
        channels = input_shape[-1] # input_shape = (batchsize, Lx, Ly, channels), and input_shape[-1] = channels
        self.conv1 = tf.keras.layers.Conv2D(channels, self.kernel_size, padding='same')
        self.conv2 = tf.keras.layers.Conv2D(channels, self.kernel_size, padding='same')
    
    def call(self, inputs):
        h = self.conv1(inputs)
        h = tf.keras.activations.relu(h)
        h = self.conv2(h)
        h = h + inputs
        return tf.keras.activations.relu(h)
    
x = tf.random.uniform(shape=[2, 28, 28, 3])
ResBlock = ResBlock1()
ResBlock(x).shape

TensorShape([2, 28, 28, 3])

実際にMNISTを想定したモデルを構築してみましょう：

In [5]:
def build_model(SkipConnection):
    '''
     MNIST用に前回作ったネットワークの畳み込み最終層に SkipConnection レイヤーを2回繰り返したモデル
    '''
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(SkipConnection()) # ここで2回 inception を繰り返す
    model.add(SkipConnection()) # ここで2回 inception を繰り返す
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.summary()
    return model

model = build_model(ResBlock1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
res_block1_1 (ResBlock1)     (None, 5, 5, 64)          73856     
_________________________________________________________________
res_block1_2 (ResBlock1)     (None, 5, 5, 64)          73856     
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0

1エポックだけ訓練してみます：

In [6]:
train(model, epochs=1)

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 7s 115us/sample - loss: 0.1253 - accuracy: 0.9623 - val_loss: 0.0374 - val_accuracy: 0.9881


ただしこのResBlockは、チャネル数 $c$ が大きくなると上の経路でずっとそのチャネル数ぶんの処理、例えばフィルターは毎回$2c^2$毎用意しなくてはならないなど、が発生するため、メモリを食ってしまう場合があります。そこで提案された二つ目のResBlockが以下です。

#### 論文のResBlockその2：ボトルネック型
アイデアは簡単で、要はチャネル数を減らしてしまえばよいのですから、

![alt](resblock5.jpg)

のように、一旦チャネル数を畳み込みの中でも一番計算量の少ない1x1畳み込みで減らし（論文では入力チャネルの1/4）、少ないチャネルで望むサイズの畳み込み演算をし、再び1x1畳み込みで元のチャネルサイズに戻すというものです：



In [7]:
class ResBlock2(tf.keras.layers.Layer):
    def __init__(self, kernel_size=(3,3)):
        super(ResBlock2, self).__init__()
        self.kernel_size = kernel_size # 
    
    def build(self, input_shape):
        channels = input_shape[-1] # input_shape = (batchsize, Lx, Ly, channels), and input_shape[-1] = channels
        self.conv_1x1_before = tf.keras.layers.Conv2D(channels//4+1, (1,1), padding='same', activation='relu')
        self.conv = tf.keras.layers.Conv2D(channels//4+1, self.kernel_size, padding='same', activation='relu')
        self.conv_1x1_after = tf.keras.layers.Conv2D(channels, (1,1), padding='same')
    
    def call(self, inputs):
        h = self.conv_1x1_before(inputs)
        h = self.conv(h)
        h = self.conv_1x1_after(h)
        h = h + inputs
        return tf.keras.activations.relu(h)
    
model = build_model(ResBlock2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
res_block2 (ResBlock2)       (None, 5, 5, 64)          4875      
_________________________________________________________________
res_block2_1 (ResBlock2)     (None, 5, 5, 64)          4875      
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)             

確かに`ResBlock1` に比べて `Param #` が減っていることがわかります。訓練もしてみましょう：

In [8]:
train(model, epochs=1)

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 8s 128us/sample - loss: 0.1384 - accuracy: 0.9565 - val_loss: 0.0470 - val_accuracy: 0.9854


#### なぜResBlockが良いのか？
深層学習では、ImageNetと呼ばれる大規模画像データを用いた画像認識コンペの最高性能(State-Of-The-Art, SOTAと略されることもあります)がネットワークの深層化とともに毎年更新されている時期で、2014年の優勝モデルの一つが次に紹介するInceptionを搭載したGoogLeNetで、22層, 誤認識率6.7%でした。これに対しResNetは2015年の優勝モデルであり、これは驚きの**152 層, 誤認識率 3.57%** でした。上に挙げたResNetの論文の冒頭で述べられていますが、あまりに深層になりすぎると、スキップ接続のないニューラルネットは深層化すると性能が悪く見えだす傾向があることが知られていました。これはAICなどの古典的なオッカムの剃刀原理に基づく素朴な期待：データのサイズに対しいたずらに巨大なモデルは汎化しない、の傍証のようにも思われるかもしれませんが、この論文では**スキップ接続がそのような深層化に伴う問題を解消する** と主張しています。

その理由はなぜか、というのは冒頭でコメントした勾配の伝搬も考えられますが、やはり理論的な決定打は無いように思われます（もしどなたかご存知なら教えて下さい！）。ここでは幾つかの論文についてコメントしておきます。

まずResBlockをベースにモデルを作ると、入力 $\to$ 出力の経路数が層数について指数的に増大していくのがわかります。各経路を別々のネットワークと考えると**アンサンブル学習** をやっているようにも思えます。この仮説は [arXiv:1605.06431](https://arxiv.org/abs/1605.06431) にて調べられ、例えば訓練後にスキップ経路を遮断すると精度が落ちるなどの傍証が確認されたようです。

また、汎化性能との兼ね合いで [**Flat minima**](https://www.mitpressjournals.org/doi/abs/10.1162/neco.1997.9.1.1) という考え方が昔からあります。これは「汎化するパラメータの周辺は経験誤差の局面がフラットになっているだろう」ということなのですが、この説もはっきり正しいとはまだわからないのですが、実際に誤差関数を2+1次元に可視化する([arXiv:1712.09913](https://arxiv.org/abs/1712.09913))とスキップ接続がある場合はない場合に比べ、誤差関数が滑らかなランドスケープを持つ傾向があるらしく、Flat minimaを思わせます。



### Inception
Inceptionは厳密に言えば上で説明したようなスキップ接続ではないのですが、似た構造を持っている有名なネットワークです。もとは[arXiv:1409.4842](https://arxiv.org/abs/1409.4842) で提案され、[arXiv:1512.00567](https://arxiv.org/abs/1512.00567)で更に改良されました。まずは一番素朴なinceptionモジュール層の構造を絵で描いてみます：

![alt](inception1.jpg)

${\color{red}{\text{conv}}}$ は畳み込み、${\color{blue}{\text{pool}}}$ はプーリングを表します。これらの演算は画像サイズを変えないようにうまく調整させます（詳しくは後述の実装参照）。$\text{concat}$ はそれぞれの経路の出力チャネルをすべてチャネル方向に統合して一つの大きなチャネルから成るテンソルと見なす操作（図では各畳込みの出力チャネル $c\_\text{out} = 2$, 入力チャネル $=3$なので、2,2,2,3チャネルの画像を 9チャネルにまとめ直す操作 ）です。

畳み込みニューラルネットワークでは、層ごとに「畳み込み演算をすべきか、プーリングすべきか」の設計を迫られる上、「それぞれのカーネルサイズをどうすべきか」なども自由度があるわけですが、inceptionがやっているのは身も蓋もない言い方をすると、**どれが良いかわからないから全部載せ** している状態です。それぞれのサブ演算で、絵ではチャネル数を全て2にして描きましたが、チャネル数を変える自由度はあります。また、この層の出力は全てのサブ演算で画像サイズを合わせます。これはゼロパディングなどで達成されます。また、プーリングは通常カーネルサイズ＝ストライドサイズ、なのですがここではプーリング結果も他のサブ演算と画像出力がおなじになるようにストライドを取ります。



In [9]:
class InceptionBlock1(tf.keras.layers.Layer):
    def __init__(self, c_out=64):
        super(InceptionBlock1, self).__init__()
        self.conv_1x1 = tf.keras.layers.Conv2D(c_out, (1, 1), padding='same', activation='relu')
        self.conv_3x3 = tf.keras.layers.Conv2D(c_out, (3, 3), padding='same', activation='relu')
        self.conv_5x5 = tf.keras.layers.Conv2D(c_out, (5, 5), padding='same', activation='relu')
        self.pool_3x3 = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1,1), padding='same')
        self.concat = tf.keras.layers.Concatenate()
    
    def call(self, inputs):
        c1 = self.conv_1x1(inputs)
        c3 = self.conv_3x3(inputs)
        c5 = self.conv_5x5(inputs)
        p3 = self.pool_3x3(inputs)
        return self.concat([c1, c3, c5, p3])

試しに以下のようなモデルにMNISTの分類をさせてみます：

In [10]:
model = build_model(InceptionBlock1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
inception_block1 (InceptionB (None, 5, 5, 256)         143552    
_________________________________________________________________
inception_block1_1 (Inceptio (None, 5, 5, 448)         573632    
_________________________________________________________________
flatten_2 (Flatten)          (None, 11200)            

まず、`c_out = 64` が一つの畳込み演算あたりの出力チャネル数なので、1x1, 3x3, 5x5の畳み込みでそれぞれ64チャネルが生成されます。従って、ここでの `naive_inception_module` の出力チャネル数は

$$
\underbrace{64 \times 3}_{192} + \# \text{input channels}
$$

になる筈です。一つ目のinceptionの入力は直前の出力の型 `Output Shape = (None, 5, 5, 64)` なので、64チャネルとわかります。従って 192+64 = 256 が一つ目のinceptionの出力チャネルとなります。これは二つ目のinceptionの入力になるが、同じ式から二つ目inceptionの出力チャネル数は 192+256 = 448 となっており、たしかに狙った操作ができている事がわかります。

1エポックだけ適当なバッチサイズで訓練してみます：

In [11]:
train(model, epochs=1)

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 8s 137us/sample - loss: 0.1192 - accuracy: 0.9625 - val_loss: 0.0326 - val_accuracy: 0.9901


#### 改良その1
ところで、上で作った素朴なInceptionモジュールは、どんどんチャネル数を増大させてゆく構造を持つため、重ねれば重ねるだけ訓練パラメータが増えて、メモリを食うのがわかります。そこでまず初めの論文 [arXiv:1409.4842](https://arxiv.org/abs/1409.4842) で提案されたのが、**更に1x1畳み込みを3x3, 5x5畳み込みの前に挿入し、そこのチャネル数を減らす**というものです。絵で描くと以下のようなモジュールを考えます：

![alt](inception2.jpg)

点線で囲った部分が素朴なinceptionと異なる部分です。3x3と5x5の畳み込み演算の直前に1x1の畳み込みを入れる理由は2つあります：
* ボトルネックによる計算量の削減
* 1x1畳み込みによる **network-in-network** 効果

まずボトルネック効果ですが、図では最初の2つの1x1畳み込みの出力チャネル数を1として描いています。こうしておくと続く畳み込み演算で容易すべきフィルターの数が減るため、ネットワークのパラメータ数が減ります。入力チャネルが巨大になっても、このように中間でチャネル数を減らしておけばチャネル数爆発しても、計算量を抑えられると期待できます。

もう一つの効果は、既に畳み込みの節でコメントしましたが、1x1畳み込みは、ピクセルに対してのミニネットワークを構成しているとも考えられます。これは [arXiv:1312.4400](https://arxiv.org/abs/1312.4400) にて提案された **network-in-network** と呼ばれる構造になっており、画像の一つの領域に多数のクラスター構造があると見つけやすい性質があるらしいです。
> 蛇足ですが inceptionとはハリウッド映画のタイトルでもあり、これには「夢の中で夢を見る」というシーンがあるのですが、network-in-networkにかけたダジャレなのでしょうか？

実装に行く前に、やはりこの場合でも出力チャネル数はそれぞれの経路で必ずしも同じ値でなくてよく、任意の値に取れることをコメントしておきます。

In [12]:
class InceptionBlock2(tf.keras.layers.Layer):
    def __init__(self, c_out=64, c_bottleneck=1):
        super(InceptionBlock2, self).__init__()
        self.conv_1x1 = tf.keras.layers.Conv2D(c_out, (1, 1), padding='same', activation='relu')
        #
        self.conv_1x1_before3x3 = tf.keras.layers.Conv2D(c_bottleneck, (1, 1), padding='same', activation='relu')
        self.conv_3x3 = tf.keras.layers.Conv2D(c_out, (3, 3), padding='same', activation='relu')
        #
        self.conv_1x1_before5x5 = tf.keras.layers.Conv2D(c_bottleneck, (1, 1), padding='same', activation='relu')
        self.conv_5x5 = tf.keras.layers.Conv2D(c_out, (5, 5), padding='same', activation='relu')
        #
        self.pool_3x3 = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1,1), padding='same')
        self.conv_1x1_afterpool = tf.keras.layers.Conv2D(c_out, (1, 1), padding='same', activation='relu')
        #
        self.concat = tf.keras.layers.Concatenate()
    
    def call(self, inputs):
        c1 = self.conv_1x1(inputs)
        c3 = self.conv_3x3(self.conv_1x1_before3x3(inputs))
        c5 = self.conv_5x5(self.conv_1x1_before5x5(inputs))
        p3 = self.conv_1x1_afterpool(self.pool_3x3(inputs))
        return self.concat([c1, c3, c5, p3])

MNIST分類させてみましょう。まず素朴なInceptionをで実験したモデルで、Inceptionモジュール部分だけ改良したバージョンのモデルを作るのは以下です。`c_bottleneck`はボトルネックのチャネル数です。絞りすぎですが、今回は計算量の削減効果をみるため `1` に取りましょう：

In [13]:
model = build_model(InceptionBlock2)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
inception_block2 (InceptionB (None, 5, 5, 256)         10754     
_________________________________________________________________
inception_block2_1 (Inceptio (None, 5, 5, 256)         35714     
_________________________________________________________________
flatten_3 (Flatten)          (None, 6400)             

素朴なInceptionを用いた場合とパラメータ数を比べると劇的に減っているのがわかります。また、出力チャネル数も減っていますが、これはプーリング経路に1x1畳み込みを追加した効果です。では1エポック訓練させましょう：

In [14]:
train(model, epochs=1)

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 10s 173us/sample - loss: 0.1346 - accuracy: 0.9579 - val_loss: 0.0496 - val_accuracy: 0.9832


#### 改良その2
改良その1が初めの論文 [arXiv:1409.4842](https://arxiv.org/abs/1409.4842) で説明された改良なのですが、この論文ではInceptionモジュールをどのように積み重ねるかのモデル（GoogLeNetと呼ばれる）の工夫も見られ、このモデルを他のサイズの画像などにも応用したい場合、調整が必要です。[arXiv:1512.00567](https://arxiv.org/abs/1512.00567) ではそのような応用に向けたさらなるInceptionモジュールの改良を行っています。ここでは僕が力尽きてきたためやりません（すいません）が、興味のある方は一読し、わかったら教えていただきたいです。